In [17]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import os
import sys

In [18]:
# Specify the root folder
root_folder = r'C:\Users\ignas\OneDrive\Documents\Postgrau_UB\17.Capstone_project'

# Walk through all subfolders and add them to sys.path
for root, dirs, files in os.walk(root_folder):
    sys.path.append(root)

from my_functions import *

In [19]:
# Load the data
# df = pd.read_csv(r'C:\Users\ignas\OneDrive\Documents\Postgrau_UB\17.Capstone_project\data\Bicing_data_merged_3.csv', nrows=0)
# Read the metadata sample submission
sample = pd.read_csv(r'C:\Users\ignas\OneDrive\Documents\Postgrau_UB\17.Capstone_project\sample\metadata_sample_submission_2025.csv')
# Read station capacity information
stations_info = pd.read_csv(r'C:\Users\ignas\OneDrive\Documents\Postgrau_UB\17.Capstone_project\data\Informacio_estacions_bicing_2025.csv')

In [20]:
time_column = 'datetime'

In [21]:
# Merge station info
sample = merge_station_info(sample,stations_info)


In [22]:

# Create a mapping for ctx-1, ctx-2, ..., ctx-4 to the new names
column_mapping = {
    'ctx-1': 'lag_1h_percentage_docks_available',
    'ctx-2': 'lag_2h_percentage_docks_available',
    'ctx-3': 'lag_3h_percentage_docks_available',
    'ctx-4': 'lag_4h_percentage_docks_available'
}

# Use the mapping to rename the columns
sample = sample.rename(columns=column_mapping)

# Add year
sample['year']=2024
day_of_week(sample)
datetime(sample)
sample[['lag_1h_is_interpolated','lag_2h_is_interpolated','lag_3h_is_interpolated','lag_4h_is_interpolated']]= False

In [23]:
sample=merge_barsa_data(sample, time_column)


In [24]:
sample=merge_weather_data(sample, time_column)

Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument


In [25]:
sample=merge_holiday_data(sample, time_column)

Index(['festiu', 'holiday'], dtype='object')


In [26]:
sample=create_lag_lead_days(sample, time_column)

In [27]:
# Find columns containing 'd_lag' or 'd_lead' in their names
lag_lead_columns = [col for col in sample.columns if 'd_lag' in col or 'd_lead' in col]

for col in lag_lead_columns:
    print(col)
    prefix = col + '_'
    sample=merge_holiday_data(sample,col, prefix=prefix)

d_lag_1d
Index(['festiu', 'd_lag_1d_holiday'], dtype='object')
d_lead_1d
Index(['festiu', 'd_lead_1d_holiday'], dtype='object')


In [28]:
sample=create_lag_lead(sample, time_column)

In [29]:
# Find columns containing 't_lag' or 't_lead' in their names
lag_lead_columns = [col for col in sample.columns if 't_lag' in col or 't_lead' in col]


for col in lag_lead_columns:
    prefix = col + '_'
    sample=merge_barsa_data(sample,col, prefix=prefix)
    sample=merge_weather_data(sample,col, prefix=prefix)

Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument
Warning during timezone localization: Cannot infer dst time from 2020-10-25 02:00:00, try using the 'ambiguous' argument


In [30]:
sample=fill_na_holiday_columns(sample)

In [31]:
# Remove any remaining datetime columns except the main time_column
sample=drop_non_time_column_dates(sample, time_column)

In [32]:
sample.to_csv("data\Metadata_prepared_v2.csv", index=True, encoding="utf-8")

In [33]:
sample.columns

Index(['index', 'station_id', 'month', 'day', 'hour',
       'lag_4h_percentage_docks_available',
       'lag_3h_percentage_docks_available',
       'lag_2h_percentage_docks_available',
       'lag_1h_percentage_docks_available', 'capacity', 'lat', 'lon',
       'altitude', 'post_code', 'year', 'day_of_week', 'datetime',
       'lag_1h_is_interpolated', 'lag_2h_is_interpolated',
       'lag_3h_is_interpolated', 'lag_4h_is_interpolated', 'FCB_Location',
       'FCB_Score', 'wea_prep', 'wea_sun', 'wea_temp', 'wea_wind_sp',
       'holiday', 'd_lag_1d_holiday', 'd_lead_1d_holiday',
       't_lag_1h_FCB_Location', 't_lag_1h_FCB_Score', 't_lag_1h_wea_prep',
       't_lag_1h_wea_sun', 't_lag_1h_wea_temp', 't_lag_1h_wea_wind_sp',
       't_lag_2h_FCB_Location', 't_lag_2h_FCB_Score', 't_lag_2h_wea_prep',
       't_lag_2h_wea_sun', 't_lag_2h_wea_temp', 't_lag_2h_wea_wind_sp',
       't_lag_3h_FCB_Location', 't_lag_3h_FCB_Score', 't_lag_3h_wea_prep',
       't_lag_3h_wea_sun', 't_lag_3h_wea_temp